In [1]:
# LIBRERÍAS
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import time

In [ ]:
# DIRECTORIOS
dir_path = ''
data_path = 'esc_primaria.csv'

In [12]:
import chardet

# Leer una muestra del archivo para detectar la codificación
with open('esc_primaria.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # Leer los primeros 10,000 bytes

# Mostrar la codificación detectada
print(result['encoding'])

ISO-8859-1


In [14]:
# CARGAR LOS DATOS
escuelas = pd.read_csv('esc_primaria.csv', encoding = 'ISO-8859-1')
escuelas.head()

,Nivel,CCT,LATITUD,LONGITUD
0,Escuela Primaria,Maria Luisa Ross,19.375208,-99.230397
1,Escuela Primaria,Enrique de Olavarría,19.376023,-99.188159
2,Escuela Primaria,Independencia,19.373755,-99.184554
3,Escuela Primaria,Maestro Ernesto Alconedo,19.381043,-99.163434
4,Escuela Primaria,Luis Cabrera,19.371972,-99.159487


In [15]:
# FUNCIÓN PARA HACER SOLICITUDES A LA API MAPBOX ISOCHRONES
def mapbox_isochrone(base_url, routing_profile, lon, lat, mins=None, meters=None, access_token=None):
    """
    Realiza una solicitud a la API Mapbox Isochrones y devuelve un GeoDataFrame con los vértices de la isócrona.
    """
    # Construye la URL de la solicitud
    if meters is not None:
        query = f"?contours_meters={meters}"
    elif mins is not None:
        query = f"?contours_minutes={mins}"
    else:
        raise ValueError("Debes especificar 'mins' o 'meters'.")
    
    url = f"{base_url}{routing_profile}/{lon},{lat}{query}&polygons=true&denoise=1&access_token={access_token}"

    # Realiza la solicitud a la API
    response = requests.get(url)
    response.raise_for_status()  # Lanza un error si la solicitud falla
    
    # Procesa los datos recibidos
    geom = response.json()["features"][0]["geometry"]["coordinates"][0]

    # Convierte las coordenadas a un DataFrame
    coords = pd.DataFrame(geom, columns=["lon", "lat"])
    return coords

In [16]:
# PRUEBA DE LA FUNCIÓN
test_coords = mapbox_isochrone(
    base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
    routing_profile="walking",
    lon=-99.184823289979, lat=19.33905855077223,
    meters=500,
    access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
)

In [17]:
print(test_coords)

          lon        lat
0  -99.185823  19.341088
1  -99.186794  19.341029
2  -99.187358  19.339593
3  -99.188514  19.339059
4  -99.189093  19.338059
5  -99.187823  19.336529
6  -99.185823  19.335395
7  -99.184823  19.335892
8  -99.183823  19.335570
9  -99.182823  19.335835
10 -99.181668  19.337059
11 -99.181906  19.339059
12 -99.182309  19.339573
13 -99.183823  19.339636
14 -99.184456  19.340426
15 -99.185823  19.341088


In [74]:
# CICLO PARA OBTENER ISÓCRONAS DE TODOS LOS CENTROS DE TRABAJO
isochrone_list = []  # Lista para guardar las isócronas
times = []  # Para estimar el tiempo de ejecución

for idx, row in escuelas.iterrows():
    start_time = time.time()
    print(f"Obteniendo isócrona para la escuela {idx + 1} de {len(escuelas)}...")

    try:
        # Obtener isócrona
        coords = mapbox_isochrone(
            base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
            #routing_profile="driving-traffic",
            routing_profile="walking",
            lon=row["LONGITUD"], lat=row["LATITUD"],
            #mins=5,  # Tiempo de conducción
            meters=1500,  # Distancia
            access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
        )
        # Convertir a polígono y añadir a la lista
        polygon = Polygon(coords.values)
        isochrone_list.append({"CCT": row["CCT"], "geometry": polygon})
    except Exception as e:
        print(f"Error al obtener la isócrona para la escuela {idx + 1}: {e}")
        continue

    # Calcular tiempo restante
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    avg_time = sum(times) / len(times)
    remaining_time = avg_time * (len(escuelas) - (idx + 1)) / 60
    print(f"Tiempo estimado restante: {remaining_time:.2f} minutos")

Obteniendo isócrona para la escuela 1 de 18...
Tiempo estimado restante: 0.08 minutos
Obteniendo isócrona para la escuela 2 de 18...
Tiempo estimado restante: 0.09 minutos
Obteniendo isócrona para la escuela 3 de 18...
Tiempo estimado restante: 0.09 minutos
Obteniendo isócrona para la escuela 4 de 18...
Tiempo estimado restante: 0.08 minutos
Obteniendo isócrona para la escuela 5 de 18...
Tiempo estimado restante: 0.08 minutos
Obteniendo isócrona para la escuela 6 de 18...
Tiempo estimado restante: 0.07 minutos
Obteniendo isócrona para la escuela 7 de 18...
Tiempo estimado restante: 0.07 minutos
Obteniendo isócrona para la escuela 8 de 18...
Tiempo estimado restante: 0.06 minutos
Obteniendo isócrona para la escuela 9 de 18...
Tiempo estimado restante: 0.05 minutos
Obteniendo isócrona para la escuela 10 de 18...
Tiempo estimado restante: 0.05 minutos
Obteniendo isócrona para la escuela 11 de 18...
Tiempo estimado restante: 0.04 minutos
Obteniendo isócrona para la escuela 12 de 18...
Tiem

In [75]:
# CONVERTIR A UN GEODATAFRAME
isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="EPSG:4326")

In [76]:
# UNIR RESULTADOS CON LA BASE ORIGINAL
escuelas_gdf = gpd.GeoDataFrame(escuelas, geometry=None)
escuelas_gdf["geometry"] = None

for idx, row in escuelas_gdf.iterrows():
    match = isochrones_gdf[isochrones_gdf["CCT"] == row["CCT"]]
    if not match.empty:
        escuelas_gdf.at[idx, "geometry"] = match["geometry"].values[0]

/tmp/ipykernel_89081/2106024614.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  escuelas_gdf["geometry"] = None


In [80]:
escuelas_gdf

,Nivel,CCT,LATITUD,LONGITUD,geometry
0,Escuela Primaria,Maria Luisa Ross,19.375208,-99.230397,"POLYGON ((-99.2284 19.37748, -99.2294 19.37748..."
1,Escuela Primaria,Enrique de Olavarría,19.376023,-99.188159,"POLYGON ((-99.18616 19.38922, -99.1874 19.3880..."
2,Escuela Primaria,Independencia,19.373755,-99.184554,"POLYGON ((-99.17855 19.38478, -99.17951 19.384..."
3,Escuela Primaria,Maestro Ernesto Alconedo,19.381043,-99.163434,"POLYGON ((-99.16043 19.3942, -99.16143 19.3945..."
4,Escuela Primaria,Luis Cabrera,19.371972,-99.159487,"POLYGON ((-99.15549 19.3853, -99.15649 19.3851..."
5,Escuela Primaria,Reino de Jordania,19.344601,-99.181292,"POLYGON ((-99.18029 19.35784, -99.18085 19.357..."
6,Escuela Primaria,No 5 Alfonso Sierra Partida,19.333262,-99.172367,"POLYGON ((-99.17537 19.34234, -99.17637 19.342..."
7,Escuela Primaria,21 de Agosto de 1944,19.342495,-99.187987,"POLYGON ((-99.18499 19.3552, -99.18799 19.3530..."
8,Escuela Primaria,Porfesor Victoriano Guzman,19.326782,-99.156576,"POLYGON ((-99.15358 19.33883, -99.15628 19.337..."
9,Escuela Primaria,Ejercito Mexicano,19.335044,-99.177859,"POLYGON ((-99.17386 19.3461, -99.17586 19.3456..."


In [77]:
# VISUALIZAR RESULTADOS CON FOLIUM
m = folium.Map(location=[19.33905855077223, -99.184823289979], zoom_start=12)

for _, row in escuelas_gdf.iterrows():
    if row["geometry"] is not None:
        folium.Polygon(
            locations=[(p[1], p[0]) for p in list(row["geometry"].exterior.coords)],
            color="blue",
            weight=1, fill_color="blue",
            fill_opacity=0.5,
            fill=True
        ).add_to(m)

In [78]:
# Guardar resultados
isochrones_gdf.to_file("primarias.shp")
escuelas_gdf.to_file("escuelas_con_isocronas.shp")

/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [79]:
# Mostrar el mapa interactivo
m.save("isocronas_escuelas.html")